In [ ]:
---
title: 1D unsteady heat transfer (3D)
description: Neumann/Dirichlet/mixed BC's and inhomogeneous BC's
author: Daning H.
show-code: False
show-prompt: False
params:
    v1:
        input: numeric
        label: Value for left Dirichlet BC, v1
        value: 0.0
        step: 1.0
    v2:
        input: numeric
        label: Value for right Dirichlet BC, v2
        value: 0.0
        step: 1.0
    prob:
        input: select
        label: BC types
        value: Dirichlet
        choices: [Dirichlet, Neumann, Mixed]
        multi: False
---

**The 3D version**

We solve a 1D unsteady heat transfer problem on the domain $0\leq x\leq L,\, t\geq 0$
$$
u_t = cu_{xx},\quad u(x,0)=f(x)
$$
where $c$ is a heat transfer constant and see figure below for the definition of initial condition $f(x)$.

The following few types of problems are solved:
+ **Dirichlet problem**: Fixed value at both ends
$$
u(0,t) = v_1,\quad u(L,t) = v_2
$$
when $v_1=v_2=0$, it is a homogeneous problem
+ **Neumann problem**: Fixed slope at both ends
$$
u_x(0,t) = 0,\quad u_x(L,t) = 0
$$
Here for simplicity, we only consider the homogeneous problem
+ **Mixed problem**: Fixed value at left end and fixed slope at right end
$$
u(0,t) = v_1,\quad u_x(L,t) = 0
$$
when $v_1=0$, it is a homogeneous problem.  Again for simplicity, we only consider zero slope.

The solutions to the above problems can be written as a series,
$$
u(x,t) = u_0(x) + \sum_{n=1}^\infty u_n(x,t)
$$
where $u_0(x)$ may be an offset due to inhomogeneity, and $u_n(x,t)$ are the eigenfunctions ("modes").  In the figures we plot the entire series solution, as well as the first five eigenfunctions.

When interacting with the different solutions, think about:
+ What is the effect of $c$?  What is the physical interpretation?
+ As $t\rightarrow\infty$, what is the solution like?  Are the modes decaying at the same rate?
+ What are the differences between the various types of problems?  What are the physical interpretations?

In [5]:
v1 = 1.0
v2 = 1.0
prob = 'Dirichlet'

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Set up domain
Nmod = 10
Nplt = 2
Nx   = 101
Nt   = 51
Nc   = 5
xx   = np.linspace(0, 2*np.pi, Nx)
tt   = np.linspace(0, 2.0, Nt).reshape(-1,1)
cs   = np.linspace(1, 8, Nc)
dt   = tt[1][0]

dats = []
sols = []
for _c in cs:
    dat = np.zeros((Nmod, Nt, Nx))
    if prob == 'Dirichlet':
        for _i in range(Nmod):
            _k = 2*_i+1
            dat[_i] = (-1)**_i * 8/np.pi/_k**2 * np.exp(-(_k*tt)**2/4*_c) * np.sin(_k*xx/2)
        inh = (v2-v1)/(2*np.pi) * xx + v1

    elif prob == 'Neumann':
        for _i in range(Nmod):
            _k = 2*_i+1
            dat[_i] = -4/np.pi/_k**2 * np.exp(-(_k*tt)**2*_c) * np.cos(_k*xx)
        inh = np.pi/2 * np.ones(Nx,)

    elif prob == 'Mixed':
        # Left Dirichlet, right Neumann
        _fct = np.sqrt(2.0)
        for _i in range(Nmod):
            _k = 2*_i+1
            _c = (-1)**(_i+1) + _fct*(-1)**(_i//2)
            dat[_i] = 16/np.pi/_k**2 * _c * np.exp(-(_k*tt)**2/4*_c) * np.sin(_k*xx/4)
        inh = v1 * np.ones_like(xx)

    sol = np.sum(dat, axis=0) + inh

    dats.append(dat)
    sols.append(sol)

# Make the plots
offset = 1+Nplt
fig = make_subplots(rows=offset, cols=1,
#                     vertical_spacing=0.15,
                    specs=[[{'type': 'surface'}]]*offset,
                    subplot_titles=("Series solution", ) + \
                    tuple([f"Mode {_i}" for _i in range(1,offset)]))
# Add traces
## Full solution
for _j in range(Nc):
    fig.add_trace(
        go.Surface(x=xx, y=tt.reshape(-1), z=sols[_j],
                   visible=False, showscale=False, hoverinfo='none'), row=1, col=1)

for _i in range(Nplt):
    for _j in range(Nc):
        fig.add_trace(
            go.Surface(x=xx, y=tt.reshape(-1), z=dats[_j][_i],
                       visible=False, showscale=False, hoverinfo='none'), row=_i+2, col=1)

## Initial visibility
for _j in range(offset):
    fig.data[_j*Nc].visible = True

# Create and add slider
steps = []
for _i in range(Nc):
    step = dict(
        method="update",
        args=[{"visible": [False] * (Nc*offset)}],
        label=f"{cs[_i]:3.2f}"
    )
    for _j in range(offset):
        step["args"][0]["visible"][_j*Nc+_i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "c = "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    height=800,
    width=800
)
fig.update_scenes(
    aspectmode='manual',
    aspectratio=dict(x=2, y=4, z=1),
    camera={
        'eye' : dict(x=0., y=0.0, z=1.0),
        'projection' : dict(type="orthographic")
    },
    xaxis_title='x (space)',
    yaxis_title='t (time)',
    zaxis_title='u (solution)')

fig.show()